In [1]:
import leafmap.plotlymap as leafmap
from jupyter_dash import JupyterDash
from dash import Dash, html,dcc,Input, Output
#import json
#import httpx

import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [ ]:
m = leafmap.Map()
m.add_basemap("Stamen.Terrain")

m.add_controls
#m.add_cog_layer(url=url,colormap_name ="viridis" )
m.show()

In [5]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'


origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File c:\Users\mdavi\anaconda3\envs\PyRapidEPE\lib\site-packages\flask\app.py:1523, in Flask.full_dispatch_request(self=<Flask '__main__'>)
   1521     rv = self.preprocess_request()
   1522     if rv is None:
-> 1523         rv = self.dispatch_request()
        rv = None
        self = <Flask '__main__'>
   1524 except Exception as e:
   1525     rv = self.handle_user_exception(e)

File c:\Users\mdavi\anaconda3\envs\PyRapidEPE\lib\site-packages\flask\app.py:1509, in Flask.dispatch_request(self=<Flask '__main__'>)
   1507     return self.make_default_options_response()
   1508 # otherwise dispatch to the handler for that endpoint
-> 1509 return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
        req = <Request 'http://127.0.0.1:8050/_dash-layout' [GET]>
        rule = <Rule '/_dash-layout' (HEAD, OPTIONS, GET) -

In [6]:
fig= container2


app = JupyterDash(__name__)

app.layout = html.Div([dcc.Graph(figure=fig)])

app.run_server(debug=True, use_reloader=False,mode="inline")  # Turn off reloader if inside Jupyter